In [10]:
# 约瑟夫环问题
# 假设有n个人， 从第一个人开始报数， 数到m的人被淘汰
# person = ['Bob', 'Jack', 'Allen', 'Tony', 'Peter', 'Rose', 'Wade']


# 约瑟夫环是一个容器
# 容器一般具有的功能有 增 删 查 改
import copy

class Person():
    def __init__(self, name, age):
        self.name = name
        self.age = age

class JosephusRing:
    def __init__(self, start, step):
        self.people = []
        self.start = start
        self.step = step

    # 增功能
    def append(self, obj):
        self.people.append(obj)

    # 删功能
    def pop(self, id_):
        self.people.pop(id_)
    
    # 查功能
    # 查找整个容器内容
    def query_list(self):
        temp = copy.copy(self.people)
        size = len(temp)
        if size == 0:
            return None
        
        index = self.start - 1
        ret = []
        for i in range(size):
            index = (index + self.step - 1) % len(temp)
            ret.append(temp.pop(index))
        return ret

    # 用生成器来进行一个一个输出 利用__next__
    def iter(self):
        temp = self.people
        start_index = self.start - 1
        while True:
            size = len(temp)
            if size == 0:
                return None
            
            index = (start_index + self.step - 1) % size
            ret = temp.pop(index)
            start_index = index
            yield ret


In [11]:
jos = JosephusRing(3, 4)
# jos.people.append(Person('Bob', 15))
# jos.people.append(Person('Jack', 12))
# jos.people.append(Person('Allen', 17))
# jos.people.append(Person('Tony', 20))
# jos.people.append(Person('Peter', 15))
# jos.people.append(Person('Rose', 16))
# jos.people.append(Person('Wade', 19))

# 通过txt文件写人Person信息
f = open('person.txt')
for line in f:
    temp = line.strip()
    name = temp[ : temp.index(',')]
    age = int(temp[temp.index(',') + 1 : ])
    jos.people.append(Person(name, age))
f.close()

# 直接查找整个容器内容
print("直接查找整个容器内容")
result = jos.query_list()
for i in range(len(result)):
    print("name:{}, age:{}" .format(result[i].name, result[i].age))

print("---------------")

jos.start = 1
result = jos.query_list()
for i in range(len(result)):
    print("name:{}, age:{}" .format(result[i].name, result[i].age))

print("---------------")

jos.pop(1)
it = jos.iter()
for per in it:
    print("name:{}, age:{}" .format(per.name, per.age))

直接查找整个容器内容
name:Rose, age:16
name:Allen, age:17
name:Bob, age:15
name:Wade, age:19
name:Jack, age:12
name:Peter, age:15
name:Tony, age:20
---------------
name:Tony, age:20
name:Bob, age:15
name:Rose, age:16
name:Peter, age:15
name:Wade, age:19
name:Allen, age:17
name:Jack, age:12
---------------
name:Peter, age:15
name:Allen, age:17
name:Bob, age:15
name:Tony, age:20
name:Wade, age:19
name:Rose, age:16
